In [12]:
import nltk as n
import pandas as pd
import nltk.tokenize
import sklearn as sk


# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

# n.download()

In [13]:
data = pd.read_csv("data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [14]:
data.describe()

,sEXT,sNEU,sAGR,sCON,sOPN,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
count,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9916.000000
mean,3.354760,2.609453,3.616643,3.474201,4.130386,429.377120,135425.315359,94.665170,3.154012,137642.476201,0.489920,0.128821
std,0.857578,0.760248,0.682485,0.737215,0.585672,428.760382,199433.803497,5.506696,311.073343,201392.066555,0.011908,0.106063
min,1.330000,1.250000,1.650000,1.450000,2.250000,24.000000,93.250000,0.040000,0.000000,0.490000,0.180000,0.000000
25%,2.710000,2.000000,3.140000,3.000000,3.750000,196.000000,16902.200000,93.770000,0.010000,17982.000000,0.490000,0.060000
50%,3.400000,2.600000,3.650000,3.400000,4.250000,317.000000,47166.900000,96.440000,0.020000,48683.000000,0.490000,0.090000
75%,4.000000,3.050000,4.150000,4.000000,4.550000,633.000000,196606.000000,97.880000,0.030000,198186.000000,0.500000,0.170000
max,5.000000,4.750000,5.000000,5.000000,5.000000,29724.900000,1251780.000000,99.820000,30978.000000,1263790.000000,0.500000,0.630000


In [30]:
# https://stackoverflow.com/questions/21633580/pandas-counting-unique-values-in-a-dataframe
unique_values = data[data.columns[0]]
pd.value_counts(unique_values.values.ravel())

e6cdef6f475cce3023c5b715f8c9f110    223
6f2bebc01062eb8334dccba3e048fdb5    219
527ed53d2ba3a3bc417b8402d5b2f556    194
d7e500ad854a1b6ced39e53a525b8a6d    184
0737e4e4980f56c9fb1cb5743001c917    172
502db2fcfe26705ae16a46c5cb2ad2e5    165
b4a21c82de4011033c8ac67081ff939c    162
b2be41464b53ffc6deae9536ddfd3aee    159
c3f4b3e345cb6b032db2e0459d179db3    153
715c9eb832dc833a0b6409ddccd268b1    151
f7456ac4e6b20911c40fdad18908a8d2    150
0bfa3d952ffed50f25011b128e73a820    141
dbdfbfda2a4205bd59b22758ceddd5af    126
e4a512374eee079d2b8acc2ce69990d5    126
f2026b8cb48aff9af31577ecbfda5c38    123
e465fadd8b30e8669f397e32e10f6cd0    118
8d7faa6d7f104a6cb7c4a9e1c6310a15    114
181962441153a36333f0c60701823412    114
d39c2b0fb2e50e37795fdbe3b8cd3792    113
eb7f8081aa0bd4004f513d3299db9063    107
521896b01c1a506dc4404e600fa99c5b    104
b7b7764cfa1c523e4e93ab2a79a946c4    102
dba5f5266d03dd6d4db084ad7dbc683c    102
3d7847b1c33b5f5811208b4aa1a7ffbd    101
c5d9ffcb242053b0abdebe0d684fea3a     99


In [32]:
line = data["STATUS"]
line.head(n=5)
#[n.word_tokenize(t) for t in n.sent_tokenize(line)]


0                          likes the sound of thunder.
1    is so sleepy it's not even funny that's she ca...
2    is sore and wants the knot of muscles at the b...
3           likes how the day sounds in this new song.
4                                          is home. <3
Name: STATUS, dtype: object

Now, let's split words and sentences of our main column.

In [ ]:
list_of_rows = []
words = []
sentences = []

list_of_rows = [l.split("\n")[0] for l in line ]


for k in list_of_rows:
    words.append(n.word_tokenize(k))
    sentences.append(n.sent_tokenize(k))
    
print("there are", len(words), "tokens and there are ", len(sentences))
print(sentences[:6])

In [34]:
fdist1 = n.FreqDist(list_of_rows)
print(fdist1)
fdist1.most_common(5)

<FreqDist with 9879 samples and 9917 outcomes>


[('black', 4),
 ('*PROPNAME*', 4),
 ('Merry Christmas everyone!', 3),
 ('No one should die because they cannot afford health care, and no one should go broke because they get sick. If you agree, please post this as your status for the rest of the day.',
  3),
 ('Happy New Year!', 3)]

In [33]:
# fdist1.hapaxes()
#set(w.lower() for w in list_of_rows)
list = [l.split("\n")[0] for l in line ]

#tokens = n.word_tokenize(list)
#tokens

#snet = n.sent_tokenize(list_of_rows)
#len(snet)

In [35]:
list_of_rows[1]

"is so sleepy it's not even funny that's she can't get to sleep."

In [38]:
print(line[1])

tag_words = n.pos_tag(words[1])
tag_words


#sent_words = n.pos_tag_sents(sentences)
#sent_words

is so sleepy it's not even funny that's she can't get to sleep.


In [ ]:
train, test = sk.cross_validation.train_test_split(data, train_size = 0.8)


def word_fea(words):
    return dict((word, True) for word in words)

#Unigram features
word_fea(words[1])


In [ ]:
# Bigram collocation
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures as BAM
from itertools import chain
from nltk.book import text4

#text4.collocations("words")

def bigram_features(words, score_fn=BAM.chi_sq): 
    bg_finder = BigramCollocationFinder.from_words(words) 
    bigrams = bg_finder.nbest(score_fn, 100000) 
    return dict((bg, True) for bg in chain(words, bigrams)) 


bigram_features(words[1], score_fn=BAM.chi_sq)